In [1]:
import lightgbm as lgb 
#from pylightgbm.models import GBMRegressor
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import time
import os
import json
import operator

import pandas as pd
import numpy as np

# 读入数据：

In [4]:
dataset= pd.read_csv("../feature/特征汇总_20170119_D.csv",encoding="gb2312") # 注意自己数据路径
from sklearn.preprocessing import MinMaxScaler
#dataset=trains[:]
#检测重复特征名称,并删除
names=dataset.columns.tolist()
print(len(names))
print(len(set(names)))
print("dataset大小：")
print(dataset.shape)
#myset = set(names)
#for item in myset: 
#    dataset.item = MinMaxScaler().fit_transform(dataset.item)
#    if names.count(item)>1:
#        print(item)
#        dataset=dataset.drop(item,axis=1)
#print("新dataset大小：")
#print(dataset.shape)

1146
1146
dataset大小：
(69495, 1146)


In [ ]:
dataset.drop('放款时间_x',axis=1, inplace=True)
dataset.drop('放款时间_y',axis=1, inplace=True)
dataset.drop('放款时间_x.1',axis=1, inplace=True)
dataset.drop('放款时间_y.1',axis=1, inplace=True)

In [83]:
tests=dataset[(dataset['标签']<0)]#测试集:13899行
trains=dataset[(dataset['标签']>=0)]#测试集:55596
trains.shape

(55596, 1142)

# 划分数据集：

In [84]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [85]:
#trains=trains.drop(['放款时间'],axis=1) #55596行  
#tests=tests.drop(['放款时间'],axis=1)#测试集:13899行
online_test=tests
train_xy,offline_test = train_test_split(trains, test_size = 0.2,random_state=21)#41697/13899
train,val = train_test_split(train_xy, test_size = 0.2,random_state=21)#27811/13886
#train,val = train_test_split(trains, test_size = 0.25,random_state=20)#41697/13899
#train_xy,offline_test = train_test_split(trains, test_size = 0.25,random_state=10)#41697/13899
train_xy.shape

(44476, 1142)

# KS评测:

In [86]:
#t1=trains.columns
#t2=tests.columns
from sklearn import metrics
def ks(y_predicted, y_true):
    label=y_true
    #label = y_true.get_label()
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()

# lightGBM训练:

In [87]:
y = train.标签
X = train.drop(['标签','用户标识'],axis=1)
#X=train[feature_list]

val_y = val.标签
val_X = val.drop(['标签','用户标识'],axis=1)
#val_X = val[feature_list]

offline_test_X=offline_test.drop(['标签','用户标识'],axis=1)
online_test_X=online_test.drop(['标签','用户标识'],axis=1)
#offline_test_X=offline_test[feature_list]
#online_test_X=online_test[feature_list]

sklearn:

In [95]:
from sklearn.model_selection import GridSearchCV

print('Start training...')
# train
gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
#gbm.fit(X, y,
#        eval_set=[(val_X, val_y)],
#        eval_metric='l1',
#        early_stopping_rounds=5)

print('Start predicting...')
# predict
#y_pred1 = gbm.predict(offline_test_X, num_iteration=gbm.best_iteration)
#y_pred2 = gbm.predict(offline_test_X, num_iteration=gbm.best_iteration)
# eval
print('Calculate feature importances...')
# feature importances
#print('Feature importances:', list(gbm.feature_importances_))

# other scikit-learn modules
estimator = lgb.LGBMRegressor(num_leaves=31)

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

gbm = GridSearchCV(estimator, param_grid)

gbm.fit(X, y)
print('Best parameters found by grid search are:', gbm.best_params_)

Start training...
Start predicting...
Calculate feature importances...


GridSearchCV(cv=None, error_score='raise',
       estimator=LGBMRegressor(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
       is_unbalance=False, label_gain=None, learning_rate=0.1, max_bin=255,
       max_depth=-1, max_drop=50, max_position=20, min_child_samples=10,
       min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=...le=1, subsample_for_bin=50000, subsample_freq=1,
       uniform_drop=False, xgboost_dart_mode=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.01, 0.1, 1], 'n_estimators': [20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [96]:
print('Best parameters found by grid search are:', gbm.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.1, 'n_estimators': 40}


In [88]:
start_time = time.time()
# create dataset for lightgbm
# if you want to re-use data, remember to set free_raw_data=False
lgb_train = lgb.Dataset(X, y, free_raw_data=False)
lgb_eval = lgb.Dataset(val_X, val_y, reference=lgb_train,free_raw_data=False)

# specify your configurations as a dict
seed=13
params2 = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    #'metric': 'binary_logloss',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.7,
    'bagging_fraction': 1,
    'bagging_freq': 10,
    'verbose': 0,
    #'num_iterations':500,
    'tree_learner':'serial',  
    'min_data_in_leaf':10,
    'feature_fraction_seed':seed,
    'bagging_seed':seed,
    'metric_freq':1 
}

print('Start training...')
# train
gbm = lgb.train(params2,
                lgb_train,
                num_boost_round=5000,
                valid_sets=lgb_eval,
                early_stopping_rounds=500)

print('Save model...')
# save model to file
gbm.save_model('../model/20170129_A.model') # 用于存储训练出的模型

Start training...
[1]	valid_0's auc:0.768512
Train until valid scores didn't improve in 500 rounds.
[2]	valid_0's auc:0.774942
[3]	valid_0's auc:0.77914
[4]	valid_0's auc:0.782765
[5]	valid_0's auc:0.784008
[6]	valid_0's auc:0.784528
[7]	valid_0's auc:0.785827
[8]	valid_0's auc:0.788563
[9]	valid_0's auc:0.789148
[10]	valid_0's auc:0.788155
[11]	valid_0's auc:0.788078
[12]	valid_0's auc:0.789031
[13]	valid_0's auc:0.788831
[14]	valid_0's auc:0.789179
[15]	valid_0's auc:0.789157
[16]	valid_0's auc:0.78979
[17]	valid_0's auc:0.790257
[18]	valid_0's auc:0.790254
[19]	valid_0's auc:0.790135
[20]	valid_0's auc:0.790994
[21]	valid_0's auc:0.791115
[22]	valid_0's auc:0.791329
[23]	valid_0's auc:0.790884
[24]	valid_0's auc:0.791329
[25]	valid_0's auc:0.791655
[26]	valid_0's auc:0.791362
[27]	valid_0's auc:0.791688
[28]	valid_0's auc:0.791473
[29]	valid_0's auc:0.792386
[30]	valid_0's auc:0.792301
[31]	valid_0's auc:0.792226
[32]	valid_0's auc:0.792069
[33]	valid_0's auc:0.792898
[34]	valid_0's

# lightGBM预测：

In [92]:
print ("跑到这里了model.predict")
preds_offline = gbm.predict(offline_test_X, num_iteration=gbm.best_iteration)#
preds_online = gbm.predict(online_test_X, num_iteration=gbm.best_iteration)#
offline=offline_test[['用户标识','标签']]
online=online_test[['用户标识','老段子特征x']]

offline['预测']=preds_offline
online['预测']=preds_online

print("线下得分;")
print(ks(offline.预测,offline.标签))

print ("跑到这里了,输出结果")
from sklearn.preprocessing import MinMaxScaler
online.预测 = MinMaxScaler().fit_transform(online.预测)
output=online[['用户标识','预测']].rename(index=str, columns={"用户标识": "userid", "预测": "probability"})
output.to_csv("../output/20170129_A.csv",index=None,encoding='utf-8')

#save feature score and feature information:  feature,score,min,max,n_null,n_gt1w
print('Calculate feature importances...')
#print('Feature importances:', list(gbm.feature_importance()))
#print('Feature importances:', list(gbm.feature_importance("gain")))
df = pd.DataFrame(X.columns.tolist(), columns=['feature'])
df['importance']=list(gbm.feature_importance())
df=df.sort(["importance"],ascending=False)
df.to_csv("../feature_score/feature_score_20170129_A.csv",index=None,encoding='gbk')
df

跑到这里了model.predict


E:\Anaconda2\envs\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda2\envs\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample

线下得分;
('ks', 0.47487943000388949)
跑到这里了,输出结果
Calculate feature importances...


E:\Anaconda2\envs\Anaconda3\lib\site-packages\ipykernel\__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,feature,importance
314,放款前浏览子行为编号_6,108
318,放款前浏览子行为编号_10,86
301,放款前浏览行为数据var,86
526,放款前信用卡额度median与放款前本期账单余额median差值,84
316,放款前浏览子行为编号_8,75
302,放款前浏览行为数据mean,69
1,1,68
641,去重后放款前上期还款金额median与放款前上期账单金额median差值,65
525,放款前上期还款金额median与放款前上期账单金额median差值,65
510,放款前上期还款金额mean与放款前上期账单金额mean差值,64
